In [ ]:
import contextily as ctx  # For basemaps
from scipy import stats
from pathlib import Path
import os
from urllib.parse import urlencode

import plotly.graph_objects as go
import plotly.express as px
from folium import plugins
import folium
import seaborn as sns
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import geopandas as gpd
import pandas as pd
import numpy as np
import warnings

import pyproj
from pyproj import CRS

os.chdir(r'C:\code\bedford-ubid')

warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

In [ ]:
parceldf = pd.read_pickle('./dataprocess/bedford_tax_process.pickle')
buildingdf = pd.read_pickle('./dataprocess/planetary_data.pickle')
## TODO change this eventually to full non-cached version.
opendf = pd.DataFrame(pd.read_pickle(r'C:\code\bedford-ubid\datasets\openmap_cache\openmap_cache.pickle'))


1545

In [96]:
len(parceldf)

parceldf

"""
"""

parceldf
type(opendf.geometry[0])

opendf

str(parceldf.geometry.iloc[0])

str(buildingdf.geometry.iloc[0])

'POLYGON ((-74.25526153715828 41.11706909559916, -74.25518939815252 41.11695571945191, -74.25509561306403 41.1169895880365, -74.25516775206978 41.117102964125245, -74.25526153715828 41.11706909559916))'

In [97]:
# parceldf['crs']

buildingdf.crs

# parceldf.crs

# buildings_reprojected = buildingdf.to_crs(parceldf.crs)

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Practical reasons:

Your parcels dataset is smaller, so it was likely already reprojected to EPSG:3857 for a specific purpose
Keeping the parcels in their current CRS avoids unnecessary computation on the smaller dataset
If you have other spatial operations planned, they're often more accurate in projected coordinates

However, EPSG:3857 has significant limitations:

Distance/area distortion: Web Mercator distorts distances and areas, especially away from the equator
Not recommended for precise spatial analysis: It's designed for web mapping, not accurate measurements

Better Option: Reproject Both to an Appropriate Local CRS

In [98]:

# For more accurate spatial analysis, use a local projected CRS
# You can find the best UTM zone or State Plane zone for your area

# Example: Find the appropriate UTM zone


# Get a sample coordinate to determine the best projection
sample_lon = buildingdf.geometry.iloc[0].centroid.x
sample_lat = buildingdf.geometry.iloc[0].centroid.y

print(f"Sample coordinates: {sample_lon}, {sample_lat}")

# # For US data, you might want to use:
# # - UTM zone (more accurate for local areas)
# # - State Plane (if staying within one state)
# # - Albers Equal Area (good for larger regions)

# # Example: Reproject both to appropriate UTM zone
# # You'll need to determine the right UTM zone for your area
# target_crs = 'EPSG:32618'  # Example: UTM Zone 18N (adjust for your location)

# buildings_proj = buildingdf.to_crs(target_crs)
# parcels_proj = parceldf.to_crs(target_crs)




Sample coordinates: -74.25517857511251, 41.11702934179897


In [99]:
# Reproject both dataframes to NY East State Plane
target_crs = 'EPSG:2262'  # NY East in feet (more common for municipal data)
# or use 'EPSG:32119' if you prefer meters

buildings_proj = buildingdf.to_crs(target_crs)
parcels_proj = parceldf.to_crs(target_crs)

# Filter buildings to parcels extent
parcels_bounds = parcels_proj.total_bounds
buildings_proj = buildings_proj.cx[
    parcels_bounds[0]:parcels_bounds[2],
    parcels_bounds[1]:parcels_bounds[3]
]

print(f"Buildings before filtering: {len(buildings_proj)}")
print(f"Buildings after filtering: {len(buildings_proj)}")

# Spatial join
result = gpd.sjoin(
    buildings_proj,
    parcels_proj,
    how='left',
    predicate='within'
)

print(f"Buildings matched to parcels: {len(result.dropna(subset=['index_right']))}")

Buildings before filtering: 12228
Buildings after filtering: 12228
Buildings matched to parcels: 9276


result

In [ ]:

parcels_proj.to_pickle('./dataprocess/parcels_projected.pickle')
buildings_proj.to_pickle('./dataprocess/buildings_projected.pickle')
